## GAN_test

In [4]:
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Reshape, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

In [16]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)
latent_dim = 100

optimizer = Adam(0.0002, 0.5)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
                     optimizer=optimizer,
                     metrics=['accuracy'])

# Build the generator
generator = build_generator()

# The generator takes noise as input and generates imgs
z = Input(shape=(latent_dim,))
img = generator(z)

# For the combined model we will only train generator
discriminator.trainable = False

# The discriminator takes generated images as input and determines validity
validity = discriminator(img)

# The combined model (stacked generator and discriminator)
# Trains the generator to fool the discriminator
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy',
                optimizer=optimizer)

W0807 15:12:49.352352  3812 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\new_ml\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0807 15:12:49.356352  3812 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\new_ml\lib\site-packages\keras\backend\tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0807 15:12:49.360352  3812 deprecation.py:323] From C:\ProgramData\Anaconda3\envs\new_ml\lib\site-packages\tensorflow\python\ops\nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0807 15:12:49.429356  3812 deprecation_wrapper.py:119] From C:\ProgramData\Anaconda3\envs\new_ml\lib\site-packages\keras\backend\tensorflow_backend.py:133: The name tf.pla

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

In [12]:
def build_generator():
    
    model = Sequential()
    
    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))
    
    model.summary()
    
    noise = Input(shape=(latent_dim,))
    img = model(noise)
    
    return Model(noise, img)

In [13]:
def build_discriminator():
    
    model = Sequential()
    
    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    img = Input(shape=img_shape)
    valid = model(img)
    
    return Model(img, valid)

In [8]:
def train(epochs, batch_size=128, sample_interval=50):
    
    # load the dataset
    (X_train, _), (_, _) = mnist.load_data()
    
    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1
    X_train = np.expand_dims(X_train, axis=3)
    
    # Adversarial ground truth
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))
    
    for epoch in range(epochs+1):
        
        # ---------------------
        # Train Discriminator
        # ---------------------
        
        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        
        # Generate a batch of new images
        gen_imgs = generator.predict(noise)
        
        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # ---------------------
        # Train Generator
        # ---------------------
        
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        
        # Train the generator (to have the discriminator label samples as valid)
        g_loss = combined.train_on_batch(noise, valid)
        
        # Plot the progress
        print ("%d [D Loss: %f, acc: %.2f%%] [G Loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))
        
        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch)

In [18]:
def sample_images(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    
    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5
    
    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("results/%d.png" % epoch)
    plt.close()

In [19]:
train(epochs=30, batch_size=32, sample_interval=200)

C:\ProgramData\Anaconda3\envs\new_ml\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D Loss: 0.625009, acc: 51.56%] [G Loss: 0.547269]
1 [D Loss: 0.435024, acc: 67.19%] [G Loss: 0.737856]
2 [D Loss: 0.368729, acc: 75.00%] [G Loss: 0.876972]
3 [D Loss: 0.261214, acc: 90.62%] [G Loss: 1.026279]
4 [D Loss: 0.231213, acc: 95.31%] [G Loss: 1.146292]
5 [D Loss: 0.198249, acc: 100.00%] [G Loss: 1.266685]
6 [D Loss: 0.161460, acc: 98.44%] [G Loss: 1.428636]
7 [D Loss: 0.163991, acc: 98.44%] [G Loss: 1.530035]
8 [D Loss: 0.144871, acc: 100.00%] [G Loss: 1.593092]
9 [D Loss: 0.145768, acc: 100.00%] [G Loss: 1.795943]
10 [D Loss: 0.110977, acc: 100.00%] [G Loss: 1.903192]
11 [D Loss: 0.095701, acc: 100.00%] [G Loss: 1.904643]
12 [D Loss: 0.084972, acc: 100.00%] [G Loss: 2.028654]
13 [D Loss: 0.075562, acc: 100.00%] [G Loss: 2.004294]
14 [D Loss: 0.078754, acc: 100.00%] [G Loss: 2.089002]
15 [D Loss: 0.069303, acc: 100.00%] [G Loss: 2.225627]
16 [D Loss: 0.075590, acc: 100.00%] [G Loss: 2.236822]
17 [D Loss: 0.053990, acc: 100.00%] [G Loss: 2.292857]
18 [D Loss: 0.065617, acc: 